In [ ]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
import base64
from dash.dependencies import Input, Output


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from ProjectOne import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "MniCia21"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))





#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Grazioso Logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Cody Gregory CS-340 Project Two'))),
    html.Hr(),
    
    ### Buttons for Filtering
    html.Div(className='row',
            style={'display' : 'flex'},
                children=[
                    html.Button(id='water-button', n_clicks=0, children='Water Rescue'),
                    html.Button(id='mountain-button', n_clicks=0, children='Mountain Rescue'),
                    html.Button(id='disaster-button', n_clicks=0, children='Disaster Rescue'),
                    html.Button(id='reset-button', n_clicks=0, children='Reset')
                    
                    #html.Button(id='water-button', waterBool=False, children='Water Rescue'),
                    #html.Button(id='mountain-button', mountBool = False, children='Mountain Rescue'),
                    #html.Button(id='disaster-button', disBool = False, children='Disaster Rescue'),
                    #html.Button(id='reset-button', resBool = False, children='Reset')
                ]),

    
   ###### 
   # Radio buttons (Show but break dashboard with callback)
   ###### 
   #html.Div(
   #     [
   #     dcc.RadioItems(
   #     id='radio_id',
   #     options=[
   #         {"label": "Water Rescue", "value": "Water"},
   #         {"label": "Mountain Rescue", "value": "Mountain"},
   #         {"label": "Disaster Rescue", "value": "Disaster"},
   #         {"label": "Reset", "value": "R"},
   #     ],
   #     labelStyle={'display': 'inline-block'},
   #     )]),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 5,
    ),
    html.Br(),
     html.Hr(),
     html.Div(className='row',
         style={'display' : 'flex'},
              children=[
                  html.Div(
                      id='graph-id',
                  className='col s12 m6',
                  
                  ),
                  html.Div(
                      id='map-id',
                  className='col s12 m6',
                  ),
                  html.Div(
                  html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))
                  )
              ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(Output('datatable-id', "data"),
    [Input('water-button', 'n_clicks'), Input('mountain-button', 'n_clicks'),
    Input('disaster-button', 'n_clicks'), Input('reset-button', 'n_clicks')])
def on_click(bt1,bt2,bt3,bt4):
    
    if (int(bt1) == 0 and int(bt2) == 0):
        df = pd.DataFrame.from_records(shelter.read({}))
    elif (int(bt1) > int(bt2)):
        df = pd.DataFrame.from_records(shelter.waterFilter({}))
    elif (int(bt2 > int(bt1))):
        df = pd.DataFrame.from_records(shelter.mountainFilter({}))
    elif (int(bt3 > int(bt1))):
        df = pd.DataFrame.from_records(shelter.disasterFilter({}))
    elif (int(bt4 > int(bt1))):
        df = pd.DataFrame.from_records(shelter.read({}))
        
    return df.to_dict('records')
    

#######
# Radio Button Logic (Breaks Dashboard just by existing)
#######
#@app.callback(
#    [Output('datatable_id', "data"),
#     Output('datatable_id', 'columns')],
#    [Input('radio_id', "value")])
#def update_filters(value):
#    
#    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#    data = df.to_dict(records)
#    
#    return (data, columns)
#    if (value == "R"):
#        df = pd.DataFrame.from_records(shelter.read({}))
#    if (value == "Water"):
#        df = pd.DataFrame.from_records(shelter.waterFilter({}))
#    if (value == "Mountain"):
#        df = pd.DataFrame.from_records(shelter.mountainFilter({}))
#    if (value == "Disaster"):
#        df = pd.DataFrame.from_records(shelter.disasterFilter({}))
#        
#    return df.to_dict('records')


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
        figure = px.pie(dff, names='breed', title='Animals')
        )
    ]


    
    
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
              dl.TileLayer(id="base-layer-id"),
              # Marker with tooltip and popup
              dl.Marker(position=[30.75,-97.48], children=[
                  dl.Tooltip(dff.iloc[0,4]),
                  dl.Popup([
                      html.H1("Animal Name"),
                      html.P(dff.iloc[1,9])
                  ])
              ])
        ])
    ]


app